In [43]:
# 4/4/2024

# Q1 i) Generate Word Context matrix for a given Corpus on a specifed window Size
#   ii) Find Similarity b/w two words using Word Content matrix

# Q2 i) Generate word vectors using word to Vec library on IMDB data set.
#   ii) Implement text classification Using Word2Vec and LSTM.

# Q3 i) Generate word embeddings using BERT library on IMDB dataset.
#   ii) Implement text classification Using BERT embeddings and BERT classifier.


In [182]:
def preprocess(corpus):
    new_corpus = []
    for i,docs in enumerate(corpus):
        corpus[i]="".join(ch for ch in docs if ch.isalnum() or ch ==' ')
        new_corpus.append([w.lower() for w in corpus[i].split()])
    return new_corpus

In [183]:
def getVocab(corpus):
    vocab = set()
    for c in corpus:
        vocab.update(c)
    v = {words:i for i,words in enumerate(vocab)}
    return v
        

In [228]:
corpus = [
    "I I like to eat apples and bananas.",
    "Apples are delicious fruits.",
    "Bananas are yellow in color.",
    "I like to code in Python."
]

In [229]:
def word_content_matrix(corpus,window_size):
    corpus = preprocess(corpus)
    vocab = getVocab(corpus)
#     print(corpus,vocab)
    
    word_list_len = len(vocab)
    
    word_context_matrix = np.zeros((word_list_len, word_list_len))
    
    for target_word,row in vocab.items():
#         print(target_word,row)
        for docs in corpus:
            for i,word1 in enumerate(docs):
                if word1 == target_word:
                    for j in range (i+1, min(i+window_size+1, len(docs))):
                        word_context_matrix[row][vocab[docs[j]]]+=1
                    for j in range (max(i-window_size,0),i):
                        word_context_matrix[row][vocab[docs[j]]]+=1
                        
    return word_context_matrix,vocab

In [230]:
word_context_matrix,vocab = word_content_matrix(corpus,2)
word_context_matrix

array([[0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 3., 2., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
       [1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 2., 1., 0., 0., 1.],
       [0., 0., 1., 2., 0., 0., 0., 0., 0., 2., 0., 1., 1., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [231]:
vocab

{'color': 0,
 'fruits': 1,
 'in': 2,
 'i': 3,
 'delicious': 4,
 'are': 5,
 'bananas': 6,
 'and': 7,
 'yellow': 8,
 'like': 9,
 'to': 10,
 'code': 11,
 'apples': 12,
 'python': 13,
 'eat': 14}

In [232]:
import pandas as pd
temp_mat = []
    
for v,i in vocab.items():
    counts = {v2: 0 for v2 in vocab.items()}
    for v2,j  in vocab.items():
        counts[v2] = word_context_matrix[i][j]
    temp_mat.append(counts)

df = pd.DataFrame(temp_mat,columns=vocab.keys(),index=vocab.keys())
df

,color,fruits,in,i,delicious,are,bananas,and,yellow,like,to,code,apples,python,eat
color,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
fruits,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
in,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
i,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0
delicious,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
are,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
bananas,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
and,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
yellow,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
like,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0


In [236]:
def word_similarity(word1, word2, vocab, word_content_matrix):
    if word1 not in vocab or word2 not in vocab:
        return "One or both words not found in the corpus."
    
    index1 = vocab[word1]
    index2 = vocab[word2]
    
    similarity = np.dot(word_content_matrix[index1], word_content_matrix[index2]) / (np.linalg.norm(word_content_matrix[index1]) * np.linalg.norm(word_content_matrix[index2]))
    
    return similarity


In [237]:
word_similarity('apples','bananas',vocab,word_context_matrix)

0.4082482904638631

In [238]:
# Q2 i) Generate word vectors using word to Vec library on IMDB data set.
#   ii) Implement text classification Using Word2Vec and LSTM.
!pip install gensim

     |████████████████████████████████| 24.1 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 35.0 MB 6.5 MB/s eta 0:00:013
  Attempting uninstall: scipy
    Found existing installation: scipy 1.6.2
    Uninstalling scipy-1.6.2:
      Successfully uninstalled scipy-1.6.2


In [246]:
!pip install nltk

In [239]:
def getPage(url):
#     url = "https://en.wikipedia.org/wiki/Taylor_Swift"
    page = urlopen(url)
    page = page.read().decode("utf-8")
    return page

In [240]:
def preProcess(page):
    soup = BeautifulSoup(page, 'html.parser')
    clean_soup = soup.get_text()
    
    whitespaces = re.compile("\n[\n]+\n")
    refs = re.compile("\[.*?\]")
    page = re.sub(whitespaces, "\n", clean_soup)
    page = re.sub(refs, " ", page)
    
    paras = ["".join(list(s.strings)).strip()+"\n" for s in soup.find_all('p')]
    paras = [s for s in paras if s != "\n"] # paras
    cleaned_text = [s for s in paras if s!="\n"]
    cleaned_text = "\n".join(cleaned_text)
    return cleaned_text[5000:5200]

In [242]:
corpus = preProcess(getPage("https://en.wikipedia.org/wiki/Taylor_Swift"))
# remove punctuations
corpus="".join(ch.lower() for ch in corpus if ch.isalnum() or ch==' ')

# corpus = [w.lower() for w in corpus.split()]
corpus

'iting an original song22 in 2003 swift and her parents started working with the talent manager dan dymtrow with his help swift modeled for abercrombie  fitch and had an original song included o'

In [247]:
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


# Preprocess the dataset
corpus = [word_tokenize(doc) for doc in dataset]

# Train Word2Vec model
model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=5, workers=4)

# Save the model
model.save("word2vec_imdb.model")

# Example usage
word = "king"
similar_words = model.wv.most_similar(word)
print(f"Words similar to '{word}': {similar_words}")


In [250]:
# Load the IMDB dataset
dataset = api.load("text8")
dataset[:10]


TypeError: 'Dataset' object is not subscriptable